# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 
import re

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt


def clean_non_western_encoded_characters_commands(text: str) -> str:
    """ Remove non-western encoded characters from a string
    List may need to grow.
    
    :param text: the text to clean
    :return: the cleaned text
    """
    text = re.sub(r"(\\begin{CJK}{UTF8}{gbsn})(.*?)(\\end{CJK})", r"\2", text)
    return text


def get_initials(name: str) -> str:
    """ Get the short name, e.g., A.-B. FamName
    :param name: full name
    :returns: initials
    """
    initials = []
    # account for non western names often in ()
    if '(' in name:
        name = clean_non_western_encoded_characters_commands(name)
        suffix = re.findall(r"\((.*?)\)", name)[0]
        name = name.replace(f"({suffix})", '')
    else:
        suffix = ''
    split = name.split()
    for token in split[:-1]:
        if '-' in token:
            current = '-'.join([k[0] + '.' for k in token.split('-')])
        else:
            current = token[0] + '.'
        initials.append(current)
    initials.append(split[-1].strip())
    if suffix:
        initials.append(f"({suffix})")
    return ' '.join(initials)

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

def robust_call(fn, value, *args, **kwargs):
    try:
        return fn(value, *args, **kwargs)
    except Exception:
        return value

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [robust_call(mpia.get_initials, k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
R. Andrae  ->  R. Andrae  |  ['R. Andrae']
S. Shahaf  ->  S. Shahaf  |  ['S. Shahaf']
M. Zhang  ->  M. Zhang  |  ['M. Zhang']
Y. Wang  ->  Y. Wang  |  ['Y. Wang']


E. Bañados  ->  E. Bañados  |  ['E. Bañados']
J. Wolf  ->  J. Wolf  |  ['J. Wolf']
S. Belladitta  ->  S. Belladitta  |  ['S. Belladitta']
Arxiv has 97 new papers today
          4 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print("Issues with the citations")
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/4 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2512.07972


extracting tarball to tmp_2512.07972...

 done.


K. El-Badry  ->  K. El-Badry  |  ['K. El-Badry']
R. Andrae  ->  R. Andrae  |  ['R. Andrae']
S. Shahaf  ->  S. Shahaf  |  ['S. Shahaf']


...

 done.
Retrieving document from  https://arxiv.org/e-print/2512.08803


extracting tarball to tmp_2512.08803...

 done.


/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found documentclass in tmp_2512.08803/main.tex

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'acronym' from 'tmp_2512.08803/acronym.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))
/opt/hostedtoolcache/Python/3.9.25/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:414: LatexWarning: Latex injecting: 'authors' from 'tmp_2512.08803/authors.tex'
  warnings.warn(LatexWarning(f"Latex injecting: '{ext}' from '{subsource}'"))


Error retrieving bib data for iso11664-4: 'iso11664-4'


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.07972-b31b1b.svg)](https://arxiv.org/abs/2512.07972) | **Chemical Signatures of AGB Mass Transfer in Gaia White Dwarf Companions**  |
|| N. Yamaguchi, et al. -- incl., <mark>K. El-Badry</mark>, <mark>R. Andrae</mark>, <mark>S. Shahaf</mark> |
|*Appeared on*| *2025-12-10*|
|*Comments*| *24 pages, 17 figures, submitted to PASP*|
|**Abstract**|            We present a homogeneous abundance analysis of 160 main-sequence stars in astrometric white-dwarf + main-sequence (WD+MS) binaries with orbits from Gaia DR3. These systems have AU-scale separations and are thought to have undergone mass transfer (MT) when the WD progenitor was an asymptotic giant branch (AGB) star. Using high-resolution spectroscopy, we measure chemical abundances of the MS stars, focusing on s-process elements. Since s-process nucleosynthesis occurs mainly in AGB stars, s-process enhancement in the MS star is a key signature of accretion from an AGB companion. We identify 40 barium dwarfs -- 36 of them newly discovered -- roughly doubling the known population in astrometric WD+MS binaries and extending it to lower metallicities than previously studied. The s-process abundances show large star-to-star variations that correlate with component masses and with metallicity but not with orbital separation. At the lowest metallicities, three barium dwarfs display strong CH and $\rm C_2$ absorption bands, linking them to CEMP-s stars and implying that AGB mass transfer usually leads to strong carbon enhancement at low metallicity. By comparing the observed abundance patterns to AGB nucleosynthesis models, we show that the diversity of s-process enhancements can be explained by variations in donor mass, metallicity, and most importantly, the number of thermal pulses the AGB star experienced before the onset of MT. Variation in the depth of the accretors' convective envelopes, with which accreted material is diluted, strengthens correlations with MS star mass and metallicity. Our results establish Gaia WD+MS binaries as a powerful laboratory for constraining binary mass-transfer physics and the origins of chemically peculiar stars.         |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.08803-b31b1b.svg)](https://arxiv.org/abs/2512.08803) | **Euclid Quick Data Release (Q1): Euclid spectroscopy of QSOs. 1. Identification and redshift determination of 3500 bright QSOs**  |
|| E. Collaboration, et al. -- incl., <mark>E. Bañados</mark>, <mark>J. Wolf</mark>, <mark>S. Belladitta</mark> |
|*Appeared on*| *2025-12-10*|
|*Comments*| *24 pages, 15 figures, 5 tables, submitted to A&A Euclid Quick Data Release (Q1) special issue*|
|**Abstract**|            The slitless spectroscopy mode of the NISP onboard Euclid has enabled efficient spectroscopy of objects within a large FoV. We present a large and homogeneous sample of bright quasars identified from the Euclid Quick Data Release (Q1) by combining high-purity candidate selections from Gaia and WISE with the NISP spectra. Through visual inspection of the Euclid spectra of these quasar candidates, we identify approximately 3500 quasars with reliable redshifts at $0<z\lesssim 4.8$. We generate the first Euclid composite spectrum of quasars covering rest-frame NUV to NIR wavelengths without telluric lines, which will be pivotal to NIR quasar spectral analysis. We obtain an empirical spectroscopic depth of $J_{\rm E}\lesssim 21.5$ and $H_{\rm E}\lesssim 21.3$ at the sensitivity of the Wide Field Survey, beyond which the number of securely identified quasars declines sharply. We analyse VIS morphologies using Sersic and CAS metrics, and a deep-learning PSF fraction to track nuclear dominance. At low redshift ($z<0.5$), obvious host structures are common and a single Sersic model fits about half of the sources; at intermediate redshift ($0.5<z<2$), the nuclear component dominates, with 90% of the Sersic fits saturating at the upper index limit. In this intermediate redshift regime, $f_{\rm PSF}$ is available, and we use it as a more reliable compactness measure than the single-Sersic and CAS parameters to quantify nuclear versus host emission. We also explore the novel Euclid NIR colour space and discuss the role of these quasars in refining AGN selection techniques for future Euclid data releases. Our results highlight the potential of Euclid spectroscopy to advance quasar surveys and enable the construction of more complete AGN catalogues. The spectroscopic bright quasar catalogue of this work, and the composite quasar spectrum, will be available at this https URL. (abridged)         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.08260-b31b1b.svg)](https://arxiv.org/abs/2512.08260) | **The Milky Way Imaging Scroll Painting Survey: Data Release 1**  |
|| J. Yang, et al. -- incl., <mark>M. Zhang</mark> |
|*Appeared on*| *2025-12-10*|
|*Comments*| *Accepted for publication in ApJS. MWISP DR1 link: this https URL*|
|**Abstract**|            We present the first data release (DR1) of the Milky Way Imaging Scroll Painting (MWISP) survey, a mapping in the J=(1-0) transition lines of 12CO, 13CO, and C18O toward the northern Galactic plane during 2011-2022. The MWISP survey was conducted using the PMO 13.7 m telescope at a spatial resolution of approximately 50" and a velocity resolution of 0.16 km/s at 115 GHz. DR1 fully covered 2310 square degrees within the Galactic longitude (l) and latitude (b) range of 9.75 deg =< l=< 229.75 deg and |b| =< 5.25 deg. The surveyed area was divided into cell units of 30'x30' for practical purposes and On-The-Fly (OTF) mapping was performed toward each target cell unit. The data were regridded into a regular 3D datacube in l-b-V_LSR with a pixel size of 30" in l-b axes and 0.16 km/s in theV_LSR axis. The median rms noise is 0.47 K, 0.25 K, and 0.25 K for 12CO, 13CO, and C18O, respectively. The equivalent 3 sigma sensitivity in 12CO luminosity is approximately 0.23 K km/s, making MWISP the most sensitive survey of its kind. In this paper, we describe the survey data, including the calibration, data cleaning, data mosaic processes, and the data products. The final mosaicked data cubes contain about 3.33x10^7 spectra (pixels) for each CO isotopologue line. Color composite images, made from the intensities of the isotopologue lines, and some concise descriptions are provided. We constructed a molecular cloud catalog based on the mosaicked 12CO data cube using the clustering algorithm DBSCAN, detecting 103,517 molecular clouds, 10,790 of which exhibit 13CO emission and 304 of which show C18O emission. Based on the histogram of voxel brightness temperature, we estimated a total 12CO flux of 7.69+/-0.38x10^7 K km/s arcmin^2, 82% of which is captured by the DBSCAN algorithm. The data, together with the cloud sample, provide unique information on molecular gas in the northern Milky Way.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2512.08324-b31b1b.svg)](https://arxiv.org/abs/2512.08324) | **Extremely energetic EUV late phase of a pair of C-class flares caused by a non-eruptive sigmoid**  |
|| <mark>Y. Wang</mark>, S. M. Mulay, L. Fletcher |
|*Appeared on*| *2025-12-10*|
|*Comments*| **|
|**Abstract**|            The EUV late phase is the second increase of the irradiance of the warm coronal lines during solar flares, and has a crucial impact on the Earth's ionosphere. In this paper, we report on the extremely energetic EUV late phase of a pair of C-class flares (SOL2012-06-17T17:26:11) observed on 2012 June 17 in NOAA active region 11504 by the \textit{Atmospheric Imaging Assembly} (AIA) instrument on board the \textit{Solar Dynamics Observatory} (SDO). The light curves integrated over the flaring region show that a factor of 4.2 more energy is released in the ``warm'' (2$-$3$\times 10^6$~K) temperature passbands (e.g. AIA 335 Å) during the late phase than during the main peaks. The origin of the emission in this extremely energetic EUV late phase is a non-eruptive sigmoid situated in a multi-polar magnetic field configuration, which is rapidly energised by C-class flares. The sigmoid plasma appears to reach temperatures in excess of $10^7$~K, before cooling to produce the EUV late-phase emission. This is seen in high-temperature passbands (e.g. AIA 131 Å) and by using differential emission measure analysis. Magnetic extrapolations indicate that the sigmoid is consistent with formation by magnetic reconnection between previously existing J-shaped loops. The sigmoid experienced a fast and a slow cooling stages, both of which were dominated by conductive cooling. The estimated total cooling time of the sigmoid is shorter than the observed value. So, we proposed that the non-eruptive sigmoid, heated by the continuous magnetic reconnection, leads to the extremely energetic EUV late phase.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2512.07972/./Figures/BaII_orbparams.png', 'tmp_2512.07972/./Figures/selection.png', 'tmp_2512.07972/./Figures/abundances_result_v2.png']
copying  tmp_2512.07972/./Figures/BaII_orbparams.png to _build/html/
copying  tmp_2512.07972/./Figures/selection.png to _build/html/
copying  tmp_2512.07972/./Figures/abundances_result_v2.png to _build/html/
exported in  _build/html/2512.07972.md
    + _build/html/tmp_2512.07972/./Figures/BaII_orbparams.png
    + _build/html/tmp_2512.07972/./Figures/selection.png
    + _build/html/tmp_2512.07972/./Figures/abundances_result_v2.png
found figures ['tmp_2512.08803/./figures/morph_vis_corner_plots_fpsf_correct.png', 'tmp_2512.08803/./figures/spec_cutouts_lowz.png', 'tmp_2512.08803/./figures/viqso_euclid_colours.png']
copying  tmp_2512.08803/./figures/morph_vis_corner_plots_fpsf_correct.png to _build/html/
copying  tmp_2512.08803/./figures/spec_cutouts_lowz.png to _build/html/
copying  tmp_2512.08803/./figures/viqso_euclid_colours.png to

## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\url}[1]{\href{#1}{#1}}$
$\newcommand{\dodoi}[1]{doi:~\href{http://doi.org/#1}{\nolinkurl{#1}}}$
$\newcommand{\doeprint}[1]{\href{http://ascl.net/#1}{\nolinkurl{http://ascl.net/#1}}}$
$\newcommand{\doarXiv}[1]{\href{https://arxiv.org/abs/#1}{\nolinkurl{https://arxiv.org/abs/#1}}}$
$\newcommand{\vdag}{(v)^\dagger}$
$\newcommand$
$\newcommand$
$\newcommand\natexlab{#1}$</div>



<div id="title">

# Chemical Signatures of AGB Mass Transfer in Gaia White Dwarf Companions

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.07972-b31b1b.svg)](https://arxiv.org/abs/2512.07972)<mark>Appeared on: 2025-12-10</mark> -  _24 pages, 17 figures, submitted to PASP_

</div>
<div id="authors">

N. Yamaguchi, et al. -- incl., <mark>K. El-Badry</mark>, <mark>R. Andrae</mark>, <mark>S. Shahaf</mark>

</div>
<div id="abstract">

**Abstract:** We present a homogeneous abundance analysis of 160 main-sequence stars in astrometric white-dwarf + main-sequence (WD+MS) binaries with orbits from Gaia DR3. These systems have AU-scale separations and are thought to have undergone mass transfer (MT) when the WD progenitor was an asymptotic giant branch (AGB) star.  Using high-resolution spectroscopy, we measure chemical abundances of the MS stars, focusing on s-process elements. Since s-process nucleosynthesis occurs mainly in AGB stars, s-process enhancement in the MS star is a key signature of accretion from an AGB companion. We identify 40 barium dwarfs -- 36 of them newly discovered -- roughly doubling the known population in astrometric WD+MS binaries and extending it to lower metallicities than previously studied. The s-process abundances show large star-to-star variations that correlate with component masses and with metallicity but not with orbital separation. At the lowest metallicities, three barium dwarfs display strong CH and $\rm C_2$ absorption bands, linking them to CEMP-s stars and implying that AGB mass transfer usually leads to strong carbon enhancement at low metallicity. By comparing the observed abundance patterns to AGB nucleosynthesis models, we show that the diversity of s-process enhancements can be explained by variations in donor mass, metallicity, and most importantly, the number of thermal pulses the AGB star experienced before the onset of MT. Variation in the depth of the accretors' convective envelopes, with which accreted material is diluted, strengthens correlations with MS star mass and metallicity. Our results establish Gaia WD+MS binaries as a powerful laboratory for constraining binary mass-transfer physics and the origins of chemically peculiar stars.

</div>

<div id="div_fig1">

<img src="tmp_2512.07972/./Figures/BaII_orbparams.png" alt="Fig9" width="100%"/>

**Figure 9. -** Barium abundance (_top_) and barium star fraction (_bottom_) as a function of several orbital and stellar parameters: (A) orbital period, (B) photocenter semi-major axis, (C) eccentricity, (D) primary mass, and (E.1, E.2) dark companion mass. Panel (E.2) is a zoom-in of panel (E.1) for companion masses below $0.85 M_{\odot}$. The pink dashed line indicates the threshold for barium dwarfs, [Ba/Fe]$=0.25 $dex. (*fig:baII_params*)

</div>
<div id="div_fig2">

<img src="tmp_2512.07972/./Figures/selection.png" alt="Fig2" width="100%"/>

**Figure 2. -** _Left_: AMRF versus luminous star mass, $M_1$, for the full non-class I sample from \citet{Shahaf2024MNRAS}(grey points). Our targets selected for follow-up are shown with colored points, separated by whether they were classified as NCE (pink diamonds) or RCE (blue circles) by \citet{Shahaf2024MNRAS}. The dotted and dashed lines are boundaries separating systems in the three classes (Section \ref{sec:selection}). The shaded regions corresponds to the expected locations of systems hosting dark compact object secondaries with masses between $0.45-0.75 M_{\odot}$(yellow, lower) and $1.4-2.1 M_{\odot}$(orange, upper), corresponding to typical WD and NS masses. While sources classified as NCE are less likely to be triples, we believe that both the NCE and RCE sources in our follow-up sample are dominated by compact object companions (see text). _Right_: Color excess against [Fe/H]\citep{Zhang2023MNRAS} for non-class I systems. The two quantities are anti-correlated, meaning that the NCE sample is biased against metal-poor primaries. We selected several RCE systems lying below the densely populated strip in this space, which are likely to host WDs. (*fig:selection*)

</div>
<div id="div_fig3">

<img src="tmp_2512.07972/./Figures/abundances_result_v2.png" alt="Fig6" width="100%"/>

**Figure 6. -** Atmospheric parameters (A) and derived abundances of various elements (B-F). In the background of each panel, we plot the distribution of the corresponding parameters for sources in the GALAH DR4 catalog. Panel (A) shows that as expected, all targets are consistent with being on the MS, with more metal-poor stars being hotter. Panels (B) - (D) show that the expected trends in the $\alpha$-elements are seen in both GALAH and our sample. We target stars that likely accreted material from an AGB donor, explaining the greater fraction of stars that are rich in s-process elements, seen in panels (E) and (F). (*fig:abundances*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.07972"></div>

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$
$\newcommand{\orcid}[1]{\orcidlink{#1}}$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand$
$\newcommand{\runin}[1]{\noindent\textbf{#1} }$</div>



<div id="title">

# Euclid Quick Data Release (Q1): $\Euclid$ spectroscopy of QSOs. 1. Identification and redshift determination of 3500 bright QSOs

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2512.08803-b31b1b.svg)](https://arxiv.org/abs/2512.08803)<mark>Appeared on: 2025-12-10</mark> -  _24 pages, 15 figures, 5 tables, submitted to A&A Euclid Quick Data Release (Q1) special issue_

</div>
<div id="authors">

E. Collaboration, et al. -- incl., <mark>E. Bañados</mark>, <mark>J. Wolf</mark>, <mark>S. Belladitta</mark>

</div>
<div id="abstract">

**Abstract:** The slitless spectroscopy mode of the $\ac{nisp}$ onboard the $\Euclid$ telescope has enabled efficient spectroscopy of objects within a large field of view. Nevertheless, the relatively low spectral resolution, overlapping spectra, and contamination pose challenges to source classification and redshift determination using the $\ac{nisp}$ spectra alone.In this work, we present a large and homogeneous sample of bright quasars identified from the Euclid Quick Data Release (Q1), constructed by combining high-purity candidate selections from $\gaia$ and WISE with the new spectroscopic capabilities of $\Euclid$ .Through visual inspection of the $\Euclid$ spectra of these quasar candidates, we identify approximately 3500 quasars with reliable redshifts in the range of $0< z \lesssim4.8$ . We generate the first $\Euclid$ composite spectrum of quasars covering rest-frame $\ac{nuv}$ to $\ac{nir}$ wavelengths without telluric lines, which will be pivotal to $\ac{nir}$ quasar spectral analysis. We obtain an empirical spectroscopic depth of $\JE \lesssim 21.5$ and $\HE \lesssim 21.3$ at the sensitivity of the Wide Field Survey, beyond which the number of securely identified quasars declines sharply. Accordingly, the sample presented in this paper comprises spectroscopically confirmed quasars brighter than these limits.We analyse VIS morphological parameters using Sérsic and model-independent (CAS) metrics, and a deep-learning $\ac{psf}$ fraction to track nuclear dominance. VIS morphologies show a clear redshift dependence: at low redshift ( $z<0.5$ ), obvious host structures are common and a single Sérsic model fits about half of the sources; at intermediate redshift ( $0.5<z<2$ ), the nuclear component dominates, with 90 \% of the Sérsic fits saturating at the upper index limit. In this intermediate redshift regime, $f_{\sfont{PSF}}$ is available, and we use it as a more reliable compactness measure than the single-Sérsic and CAS parameters to quantify nuclear versus host emission. We also explore the novel $\Euclid$ $\ac{nir}$ colour space and discuss the role of these quasars in refining $\ac{agn}$ selection techniques for future $\Euclid$ data releases. Our results highlight the potential of $\Euclid$ spectroscopy to advance quasar surveys and enable the construction of more complete $\ac{agn}$ catalogues.The spectroscopic bright quasar catalogue of this work, and the composite quasar spectrum, will be available at $\url{https://cdsarc.cds.unistra.fr/}$ .

</div>

<div id="div_fig1">

<img src="tmp_2512.08803/./figures/morph_vis_corner_plots_fpsf_correct.png" alt="Fig12" width="100%"/>

**Figure 12. -** 
Same as \cref{fig:morph_vis_corner_lowz}, but for 2361 sources at $0.5<z<2$ with measurements of \ac{agn}\ac{psf} contribution fraction ($f_{\sfont{PSF}}$).
 (*fig:morph_vis_corner_fpsf*)

</div>
<div id="div_fig2">

<img src="tmp_2512.08803/./figures/spec_cutouts_lowz.png" alt="Fig14" width="100%"/>

**Figure 14. -** \Euclid\ac{1d} spectra and imaging cutouts for a random sample of 10 low-redshift ($z<0.5$) sources.
Each panel displays the \ac{1d} spectrum (top) and five imaging cutouts with $10"$ sizes (bottom) in the \IE, \YE, \JE, and \HE bands, as well as a VIS-\YE composite. The composite image is generated by mapping the VIS and \YE fluxes into the blue and red channels, respectively, with their mean used for green, and the VIS band used to define overall luminosity in the L*a*b* colour space  (iso11664-4)  to enhance morphological detail. Major emission lines detected in the wavelength range [12 047, 18 734] Å are marked.  (*fig:cutouts-lowz*)

</div>
<div id="div_fig3">

<img src="tmp_2512.08803/./figures/viqso_euclid_colours.png" alt="Fig7" width="100%"/>

**Figure 7. -** \Euclid and external \citep[Hyper Suprime-Cam, HSC;][]{2018PASJ...70S...1M,2018PASJ...70S...4A} colour-colour diagrams of spectroscopically identified quasars in this work, where \gaia-detected sources are shown as blue open squares, and sources not in \gdr{3} are shown as orange circles. The density distribution of the \gaia-detected sample is indicated with blue contour lines, and that of the non-\gaia sample is indicated with red contour lines. The density of \Euclid point-like sources is shown as grey hexagonal binning plots in the background. (*fig:euclid_ccds*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2512.08803"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

140  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

10  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

4  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
